In [ ]:
# v2: 해상도 높이기

In [ ]:
# v3: 마커 여러 개 인식하고 id와 면적 (픽셀) 출력하기

In [1]:
import cv2
import cv2.aruco as aruco

# 카메라 열기
cap = cv2.VideoCapture(1)  # 0: 기본 카메라, 필요 시 1로 변경
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # 해상도 설정
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# ArUco 마커 딕셔너리 설정 (4x4 마커, 50개)
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
aruco_params = aruco.DetectorParameters()

# OpenCV 4.7 이상 버전 감지기 설정
detector = aruco.ArucoDetector(aruco_dict, aruco_params)

def find_aruco_markers(img, draw=True):
    """ ArUco 마커를 감지하고 ID 및 면적을 반환하는 함수 """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 흑백 변환
    corners, ids, _ = detector.detectMarkers(gray)  # 마커 감지
    
    marker_info = []  # 마커 정보 저장 (ID, 면적)

    if ids is not None:
        for i, corner in enumerate(corners):
            marker_id = ids[i][0]  # 마커 ID
            
            # 사각형의 네 개 좌표 가져오기
            x1, y1 = corner[0][0]
            x2, y2 = corner[0][1]
            x3, y3 = corner[0][2]
            x4, y4 = corner[0][3]

            # 다각형 면적 계산 (픽셀 단위)
            area = 0.5 * abs((x1*y2 + x2*y3 + x3*y4 + x4*y1) - (y1*x2 + y2*x3 + y3*x4 + y4*x1))
            
            marker_info.append((marker_id, area))  # ID와 면적 저장
            
            if draw:
                aruco.drawDetectedMarkers(img, [corner])  # 마커 표시
                cv2.putText(img, f"ID: {marker_id}, Area: {int(area)}", 
                            (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    return marker_info

while True:
    ret, frame = cap.read()
    if not ret:
        print("카메라 프레임을 읽을 수 없습니다.")
        break

    markers = find_aruco_markers(frame)

    for marker in markers:
        print(f"마커 ID: {marker[0]}, 면적: {int(marker[1])} 픽셀^2")

    # 화면 출력
    cv2.imshow("ArUco Detection", frame)

    # 'q' 또는 ESC 키로 종료
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:
        break

cap.release()
cv2.destroyAllWindows()
